In [1]:
#Anna Wojciechowska, july, Oslo 2020
import xml.etree.ElementTree as et

import ijson
import json

import pandas as pd
import numpy as np

import time

parse coverity xml and limit to **qtbase** only

In [10]:
xtree = et.parse("coverityReport.xml")
xroot = xtree.getroot()

attributes = list(xroot)[0].attrib
dfCols = []
rows = []
for atr in attributes:
    dfCols.append(atr)
#tutaj    
for node in xroot:
    nodeElems = node.attrib
    row = {}    
    for key, value in nodeElems.items():
        row[key] = value
    rows.append(row)
    
coverityDF = pd.DataFrame(rows, columns = dfCols)
print("total number of bugs: " + str(len(coverityDF)))
coverityDFQtBase = coverityDF[coverityDF['displayComponent']=='qtbase']
print("number of qtbase bugs: " + str(len(coverityDFQtBase)))
print("number of distinct files with bugs from qtbase: " + str(coverityDFQtBase['file'].nunique()))
# apparently there are no duplicates coverityDFQtBase.drop_duplicates()
coverityDFQtBase.head()

total number of bugs: 1117
number of qtbase bugs: 477
number of disting files with bugs from qtbase: 295


,cid,type,impact,status,firstDetected,owner,classification,severity,action,displayComponent,category,file,function
49,266725,Uninitialized pointer field,Medium,New,02/23/20,Unassigned,Unclassified,Unspecified,Undecided,qtbase,Uninitialized members,/qtbase/src/plugins/sqldrivers/mysql/qsql_mysq...,QMYSQLResultPrivate
65,265543,Uninitialized scalar field,Medium,New,01/26/20,Unassigned,Unclassified,Unspecified,Undecided,qtbase,Uninitialized members,/qtbase/src/corelib/time/qdatetimeparser_p.h,QDateTimeParser
74,264511,Dereference null return value,Medium,New,01/02/20,Unassigned,Unclassified,Unspecified,Undecided,qtbase,Null pointer dereferences,/qtbase/src/widgets/widgets/qtabbar.cpp,paintEvent
75,264510,Dereference null return value,Medium,New,01/02/20,Unassigned,Unclassified,Unspecified,Undecided,qtbase,Null pointer dereferences,/qtbase/src/widgets/widgets/qtabbar.cpp,selectNewCurrentIndexFrom
80,263964,Dereference after null check,Medium,New,11/10/19,Unassigned,Unclassified,Unspecified,Undecided,qtbase,Null pointer dereferences,/qtbase/src/widgets/itemviews/qabstractitemvie...,selectionCommand


parsing code checker report

In [ ]:
start = time.time()
#codechecker
codeCheckerDF = pd.DataFrame(columns=('category', 'check_name', 'description', 'hash', 'type','line_no' , 'file'))
it = iter (ijson.parse(open("codeCheckerReport.json")))

for i in range(10000):
    skipping = True
    while (skipping):
        item = next(it)
        first_item = item[0]
        if first_item == "item.category":
            category = item[2]
            continue
        
        if first_item == "item.check_name":
            check_name = item[2]
            continue
            
        if first_item == "item.description":
            description = item[2]
            continue
        
        if first_item == "item.issue_hash_content_of_line_in_context":
            hashValue = item[2]
            continue
            
        if first_item == "item.location.file":
            file_loc = int(item[2])
            continue
            
        if first_item == "item.location.line":
            line_no = int(item[2])
            continue
            
        if first_item == "item.type":
            checkerType = item[2]
            continue
        
        if first_item=="item.files" and item[1] == "start_array":
            filesFields = True
            fileList = []
            while (filesFields):
                item = next(it)
                filePath = "None" if item[2] is None else str(item[2])
                #"normalize path to make them comparable with coverity report"
                if filePath.startswith("/home/qt/qtCode/"):
                    filePath = filePath[15:]
                fileList.append(filePath)
                    
                if item [0] =="item.files" and item[1] == "end_array":
                    skipping = False
                    file = fileList[file_loc]
                    newRow = {'category':category, 'check_name':check_name, 'description': description, 'hash':hashValue, 'type':checkerType, 'line_no': line_no, 'file':file}
                    codeCheckerDF = codeCheckerDF.append(newRow, ignore_index=True)
                    filesFields = False
                    #codeCheckerDF = codeCheckerDF.drop_duplicates()

codeCheckerDF = codeCheckerDF.drop_duplicates()
end = time.time()

print("Number of bugs read from codechecker: " + str(len(codeCheckerDF)))
print(end - start)

code checker has a separate files for mapping severity of the bug

In [46]:
with open('checker_severity_map.json','r') as f:
    severity_map = json.loads(f.read())

codeCheckerDF["severity"] = ""

for i, row in codeCheckerDF.iterrows():
    try:
        severity = severity_map[codeCheckerDF.check_name[i]]
    except KeyError:
        severity = ""
    row['severity'] = severity

Join 2 data frames from coverity and codechecker based on file name

how{‘left’, ‘right’, ‘outer’, ‘inner’}, default ‘inner’
Type of merge to be performed.

left: use only keys from left frame, similar to a SQL left outer join; preserve key order.

right: use only keys from right frame, similar to a SQL right outer join; preserve key order.

outer: use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically.

inner: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.

In [47]:
merged = codeCheckerDF.merge(coverityDFQtBase,how='inner',left_on=['file'],right_on=["file"])
print(len(merged))
merged.head()

284


,category_x,check_name,description,hash,type_x,line_no,file,severity_x,cid,type_y,impact,status,firstDetected,owner,classification,severity_y,action,displayComponent,category_y,function
0,cppcoreguidelines,cppcoreguidelines-special-member-functions,"class 'AbstractDebugStreamFunction' defines a copy constructor and a copy assignment operator but does not define a destructor, a move constructor or a move assignment operator",9db9af407b3d7342a749bc13fd59136b,clang-tidy,256,/qtbase/src/corelib/kernel/qmetatype.h,LOW,168117,Missing move assignment operator,Low,New,07/31/16,Unassigned,Unclassified,Unspecified,Undecided,qtbase,Low impact quality,
1,cppcoreguidelines,cppcoreguidelines-special-member-functions,"class 'AbstractComparatorFunction' defines a copy constructor and a copy assignment operator but does not define a destructor, a move constructor or a move assignment operator",ced2e8973974a08e387b8e8725b8a6f8,clang-tidy,285,/qtbase/src/corelib/kernel/qmetatype.h,LOW,168117,Missing move assignment operator,Low,New,07/31/16,Unassigned,Unclassified,Unspecified,Undecided,qtbase,Low impact quality,
2,cppcoreguidelines,cppcoreguidelines-special-member-functions,"class 'AbstractConverterFunction' defines a copy constructor and a copy assignment operator but does not define a destructor, a move constructor or a move assignment operator",8db9a2f0c0e817dee578c6643737c8f0,clang-tidy,341,/qtbase/src/corelib/kernel/qmetatype.h,LOW,168117,Missing move assignment operator,Low,New,07/31/16,Unassigned,Unclassified,Unspecified,Undecided,qtbase,Low impact quality,
3,cppcoreguidelines,cppcoreguidelines-special-member-functions,"class 'ConverterMemberFunction' defines a non-default destructor but does not define a copy constructor, a copy assignment operator, a move constructor or a move assignment operator",bef1b2dbd543157eea27c6f9d8260cc5,clang-tidy,351,/qtbase/src/corelib/kernel/qmetatype.h,LOW,168117,Missing move assignment operator,Low,New,07/31/16,Unassigned,Unclassified,Unspecified,Undecided,qtbase,Low impact quality,
4,cppcoreguidelines,cppcoreguidelines-special-member-functions,"class 'ConverterMemberFunctionOk' defines a non-default destructor but does not define a copy constructor, a copy assignment operator, a move constructor or a move assignment operator",d03ac1274553104f4525642b3c6a3325,clang-tidy,371,/qtbase/src/corelib/kernel/qmetatype.h,LOW,168117,Missing move assignment operator,Low,New,07/31/16,Unassigned,Unclassified,Unspecified,Undecided,qtbase,Low impact quality,


In [17]:
print(merged['file'].nunique())

15


In [35]:
#merged = merged.dropna(thresh=1)
merged = merged.dropna(subset=['cid', 'line_no'])
compareDF = merged[['severity_x' ,'impact', 'file','check_name', 'description' , 'type_y',  'type_x','category_x']]
print(len(compareDF))
print("Distinct files: " + str( len(compareDF.file.unique())))
compareDF.head()

187
Distinct files: 15


,severity_x,impact,file,check_name,description,type_y,type_x,category_x
0,LOW,Low,/qtbase/src/corelib/kernel/qmetatype.h,cppcoreguidelines-special-member-functions,class 'AbstractDebugStreamFunction' defines a ...,Missing move assignment operator,clang-tidy,cppcoreguidelines
1,LOW,Low,/qtbase/src/corelib/kernel/qmetatype.h,cppcoreguidelines-special-member-functions,class 'AbstractComparatorFunction' defines a c...,Missing move assignment operator,clang-tidy,cppcoreguidelines
2,LOW,Low,/qtbase/src/corelib/kernel/qmetatype.h,cppcoreguidelines-special-member-functions,class 'AbstractConverterFunction' defines a co...,Missing move assignment operator,clang-tidy,cppcoreguidelines
3,LOW,Low,/qtbase/src/corelib/kernel/qmetatype.h,cppcoreguidelines-special-member-functions,class 'ConverterMemberFunction' defines a non-...,Missing move assignment operator,clang-tidy,cppcoreguidelines
4,LOW,Low,/qtbase/src/corelib/kernel/qmetatype.h,cppcoreguidelines-special-member-functions,class 'ConverterMemberFunctionOk' defines a no...,Missing move assignment operator,clang-tidy,cppcoreguidelines


In [36]:
compareDF = compareDF.drop_duplicates()
print("Distinct: " + str( len(compareDF)))

Distinct: 113


In [37]:
compareDF

,severity_x,impact,file,check_name,description,type_y,type_x,category_x
0,LOW,Low,/qtbase/src/corelib/kernel/qmetatype.h,cppcoreguidelines-special-member-functions,class 'AbstractDebugStreamFunction' defines a ...,Missing move assignment operator,clang-tidy,cppcoreguidelines
1,LOW,Low,/qtbase/src/corelib/kernel/qmetatype.h,cppcoreguidelines-special-member-functions,class 'AbstractComparatorFunction' defines a c...,Missing move assignment operator,clang-tidy,cppcoreguidelines
2,LOW,Low,/qtbase/src/corelib/kernel/qmetatype.h,cppcoreguidelines-special-member-functions,class 'AbstractConverterFunction' defines a co...,Missing move assignment operator,clang-tidy,cppcoreguidelines
3,LOW,Low,/qtbase/src/corelib/kernel/qmetatype.h,cppcoreguidelines-special-member-functions,class 'ConverterMemberFunction' defines a non-...,Missing move assignment operator,clang-tidy,cppcoreguidelines
4,LOW,Low,/qtbase/src/corelib/kernel/qmetatype.h,cppcoreguidelines-special-member-functions,class 'ConverterMemberFunctionOk' defines a no...,Missing move assignment operator,clang-tidy,cppcoreguidelines
...,...,...,...,...,...,...,...,...
179,LOW,Medium,/qtbase/src/corelib/kernel/qobject_p.h,cppcoreguidelines-special-member-functions,class 'QMetaCallEvent' defines a non-default d...,Uninitialized pointer field,clang-tidy,cppcoreguidelines
181,LOW,Medium,/qtbase/src/corelib/kernel/qobject_p.h,cppcoreguidelines-special-member-functions,class 'QDynamicMetaObjectData' defines a non-d...,Improper use of negative value,clang-tidy,cppcoreguidelines
182,LOW,Medium,/qtbase/src/corelib/kernel/qobject_p.h,cppcoreguidelines-special-member-functions,class 'QDynamicMetaObjectData' defines a non-d...,Uninitialized pointer field,clang-tidy,cppcoreguidelines
184,LOW,Medium,/qtbase/src/corelib/kernel/qobject_p.h,cppcoreguidelines-special-member-functions,class 'QAbstractDynamicMetaObject' defines a n...,Improper use of negative value,clang-tidy,cppcoreguidelines


In [38]:
miniDF = compareDF[['severity_x' ,'impact', 'file', 'description' , 'type_y', ]]


In [42]:
pd.set_option('display.max_colwidth', None)
miniDF

,severity_x,impact,file,description,type_y
0,LOW,Low,/qtbase/src/corelib/kernel/qmetatype.h,"class 'AbstractDebugStreamFunction' defines a copy constructor and a copy assignment operator but does not define a destructor, a move constructor or a move assignment operator",Missing move assignment operator
1,LOW,Low,/qtbase/src/corelib/kernel/qmetatype.h,"class 'AbstractComparatorFunction' defines a copy constructor and a copy assignment operator but does not define a destructor, a move constructor or a move assignment operator",Missing move assignment operator
2,LOW,Low,/qtbase/src/corelib/kernel/qmetatype.h,"class 'AbstractConverterFunction' defines a copy constructor and a copy assignment operator but does not define a destructor, a move constructor or a move assignment operator",Missing move assignment operator
3,LOW,Low,/qtbase/src/corelib/kernel/qmetatype.h,"class 'ConverterMemberFunction' defines a non-default destructor but does not define a copy constructor, a copy assignment operator, a move constructor or a move assignment operator",Missing move assignment operator
4,LOW,Low,/qtbase/src/corelib/kernel/qmetatype.h,"class 'ConverterMemberFunctionOk' defines a non-default destructor but does not define a copy constructor, a copy assignment operator, a move constructor or a move assignment operator",Missing move assignment operator
...,...,...,...,...,...
179,LOW,Medium,/qtbase/src/corelib/kernel/qobject_p.h,"class 'QMetaCallEvent' defines a non-default destructor but does not define a copy constructor, a copy assignment operator, a move constructor or a move assignment operator",Uninitialized pointer field
181,LOW,Medium,/qtbase/src/corelib/kernel/qobject_p.h,"class 'QDynamicMetaObjectData' defines a non-default destructor but does not define a copy constructor, a copy assignment operator, a move constructor or a move assignment operator",Improper use of negative value
182,LOW,Medium,/qtbase/src/corelib/kernel/qobject_p.h,"class 'QDynamicMetaObjectData' defines a non-default destructor but does not define a copy constructor, a copy assignment operator, a move constructor or a move assignment operator",Uninitialized pointer field
184,LOW,Medium,/qtbase/src/corelib/kernel/qobject_p.h,"class 'QAbstractDynamicMetaObject' defines a non-default destructor but does not define a copy constructor, a copy assignment operator, a move constructor or a move assignment operator",Improper use of negative value
